In [14]:
print('=====================================================')
print('          Welcome to MovieMatch Recommender! ')
print('     Your personalized movie suggestions start here.')
print('          Rate a few movies to get started 🎥')
print('=====================================================')

          Welcome to MovieMatch Recommender! 
     Your personalized movie suggestions start here.
          Rate a few movies to get started 🎥


### Load data ###

In [31]:
import os
import csv

USERS_FILE = 'Data/users_map.csv'
MOVIES_FILE = "Data/u.item"
USERS_RATING = 'Data/u.data.csv'

def load_user(username, password):
    if not os.path.exists(USERS_FILE):
        return None
    
    with open(USERS_FILE, "r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row["username"] == username and row["password"] == password:
                return {
                    "user_id": row["user_id"],
                    "username": row["username"]
                }
    return None


def load_movies():
    movies = {}
    with open(MOVIES_FILE, "r", encoding="latin-1") as f:
        for line in f:
            parts = line.strip().split("|")
            movie_id = parts[0]
            title = parts[1]
            movies[movie_id] = title
    return movies

movies_dict = load_movies()

movie_name_to_id = load_movies()

### Load user ratings ###

In [32]:
def load_users_rating(user_id):
    """
    Returns a list of dictionaries like:
    [{'movie_id': '86', 'rating': '3'}, ...]
    """
    if not os.path.exists(USERS_RATING):
        return []

    ratings = []
    with open(USERS_RATING, "r", encoding="utf-8") as file:
        next(file)  # skip header if exists
        for line in file:
            parts = line.strip().split()  # جداکننده space/tab
            if parts[0] == str(user_id):
                ratings.append({
                    "movie_id": parts[1],
                    "rating": parts[2]
                })
    return ratings

    

### Print ratings ###

In [38]:
def print_rating(movie_id):
    
    """
    Returns the movie title for a given movie_id from u.item.
    Output: {'movie_id': id, 'title': movie title} or None if not found.
    """
    
    if not os.path.exists(MOVIES_FILE):
        return None

    try:
        with open(MOVIES_FILE, "r", encoding="latin-1") as file:
            for line in file:
                parts = line.strip().split("|")
                if parts[0] == str(movie_id):
                    return {
                        "movie_id": parts[0],
                        "title": parts[1]
                    }
    except:
        return None


### Save new user to csv file ###

In [34]:
def save_new_user(username, password):
    next_id = 1
    existing_data = []
    if os.path.exists(USERS_FILE):
        with open(USERS_FILE, 'r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            existing_data = list(reader)
            ids = [int(row['user_id']) for row in existing_data if 'user_id' in row]
            if ids:
                next_id = max(ids) + 1

    with open(USERS_FILE, 'w', newline='', encoding='utf-8') as file:
        fieldnames = ['user_id', 'username', 'password']
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        if existing_data:
            writer.writeheader()
            for row in existing_data:
                if all(key in row for key in fieldnames):
                    writer.writerow(row)

        writer.writerow({
            'user_id': next_id,
            'username': username,
            'password': password
        })

    return next_id

### Save user rating to csv file ###

In [35]:
def save_users_rating(user_id, movie_id, rating):
    new_row = {
        "user_id": str(user_id),
        "movie_id": str(movie_id),
        "rating": str(rating)
    }

    ratings = []
    if os.path.exists(USERS_RATING):
        with open(USERS_RATING, 'r', newline='', encoding='utf-8') as file:
            next(file)  # skip header
            for line in file:
                parts = line.strip().split()
                ratings.append({
                    "user_id": parts[0],
                    "movie_id": parts[1],
                    "rating": parts[2]
                })

    ratings.append(new_row)

    with open(USERS_RATING, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["user_id", "movie_id", "rating"])
        writer.writeheader()
        writer.writerows(ratings)

    print("Rating saved successfully.")

### Main program (ask user if already have an account) ###

In [36]:
while True:
    answer = input("Do you already have an account? (yes/no): ").strip().lower()
    if answer in ['yes', 'no']:
        break
    print("Invalid input. Please type 'yes' or 'no'.\n")


### Login section with displaying user's ratings ###

In [37]:
if answer == "yes":
    print("\n--- LOGIN ---")
    username = input("Enter your username: ").strip()
    password = input("Enter your password: ").strip()

    user = load_user(username, password)
    if user is None:
        print("❌ Username or password is incorrect.")
    else:
        print(f"\n✅ Login successful! Welcome {user['username']}")
        print(f"Your user ID is: {user['user_id']}")

        # --- Load user ratings using load_users_rating ---
        user_ratings = load_users_rating(user['user_id'])

        if user_ratings:
            print("\n🎬 Your rated movies:")
            print("{:<6} {:<50} {:<6}".format("ID", "Title", "Rating"))
            print("-"*70)
            for r in user_ratings:
                movie_id = r['movie_id']
                title = movies_dict.get(movie_id, "Unknown")
                print("{:<6} {:<50} {:<6}".format(movie_id, title, r['rating']))
        else:
            print("\nYou haven't rated any movies yet.")


--- LOGIN ---

✅ Login successful! Welcome chris
Your user ID is: 15

🎬 Your rated movies:
ID     Title                                              Rating
----------------------------------------------------------------------
405    Mission: Impossible (1996)                         2     
749    MatchMaker, The (1997)                             1     
25     Birdcage, The (1996)                               3     
331    Edge, The (1997)                                   3     
222    Star Trek: First Contact (1996)                    3     
473    James and the Giant Peach (1996)                   1     
678    Volcano (1997)                                     1     
932    First Kid (1996)                                   1     
127    Godfather, The (1972)                              2     
685    Executive Decision (1996)                          4     
20     Angels and Insects (1995)                          3     
301    In & Out (1997)                                   

### Create new account section ###

In [13]:
if answer == "no":
    print("\n--- CREATE NEW ACCOUNT ---")

    new_username = input("Choose a username: ").strip()
    new_password = input("Choose a password: ").strip()

    new_id = save_new_user(new_username, new_password)

    print(f"\n✅ Account created successfully!")
    print(f"Your username: {new_username}")
    print(f"Your user ID: {new_id}")
    
    while True:
        print('Do you want to rate your own movies? yes/no')
        ans = input('Do you want to rate your own movies?').strip().lower()
        if ans == 'yes':
            movie_name = input("Enter your movie's name :").strip()
            movie_rate = input("Enter your movie's rating").strip()
            
            movie_id = movie_name_to_id.get(movie_name)
            
            if movie_id is None:
                print("❌ Movie not found in dataset.")
                
            else:
                print(f"Movie found! ID = {movie_id}")
                print("Saving to user ratings...")
                
                save_users_rating(new_id, movie_name, movie_rate)
                
            break
        if ans == 'no':
            break


--- CREATE NEW ACCOUNT ---

✅ Account created successfully!
Your username: asgharx
Your user ID: 2505
Do you want to rate your own movies? yes/no
Movie found! ID = 11
Saving to user ratings...
Rating saved successfully.
